# Scraping

#### Look at the Links

In [ ]:
from bs4 import BeautifulSoup # the library that provides the HTTP calls etc.
from urllib.request import urlopen


url = "https://ai.google/research/pubs/?year=2019" # The url we want to scrape
raw = urlopen(url).read() # open the page and get the html
soup = BeautifulSoup(raw ,"html.parser") # parse html
links = [l.get("href") for l in soup.find_all("a")] # identify the links
links = [l for l in links if (l is not None)] # filter the empty links


links

In [ ]:
j=47700 
paperLinks = []

for i in range(30):
    paperLinks.append("https://ai.google/research/pubs/pub" + str(j))
    j=j+1
    
    
paperLinks

#### Get the titles and abstracts

In [ ]:
from bs4 import BeautifulSoup
import urllib.request
title = list()
abstract = list()

import ssl

ssl._create_default_https_context = ssl._create_unverified_context

for l in paperLinks:
    try:
        html = urllib.request.urlopen(l)
    except Exception as e:
        print(e)
    else:
        for l in paperLinks:
            raw = html.read() 
            soup = BeautifulSoup(raw) 
            title = title + [t.string.strip() for t in soup.find_all("h1", class_="meta--publication__title")]
            try:
                abstract = abstract + [a.string.strip() for a in soup.find_all("p")]
            except Exception as e:
                print(e)
                continue

In [ ]:
abstract

In [ ]:
papers = list(zip(title, abstract))
print(papers)

In [ ]:

words = [ w.lower() for t in abstract for w in t.split() ]

words[:10]

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
stop = nltk.corpus.stopwords.words('english')
words_without_stopwords = [i for i in words if i not in stop]
print (words_without_stopwords[:10])

In [ ]:
from nltk.collocations import *
bigram_measures = nltk.collocations.BigramAssocMeasures()


raw = " ".join(abstract)
tokens = nltk.word_tokenize(raw)
text = nltk.Text(tokens)

finder = BigramCollocationFinder.from_words(text)
finder.nbest(bigram_measures.pmi, 10)

In [ ]:
from prettytable import PrettyTable
from collections import Counter 

porter = nltk.PorterStemmer()
text_stemmed = [porter.stem(t).lower() for t in tokens if t not in stop]

pt = PrettyTable(field_names=['Stem', 'Frequency']) 
c = Counter(text_stemmed)
[ pt.add_row(kv) for kv in c.most_common()[:20] ]
pt.align['Wortstamm'], pt.align['Häufigkeit'] = 'l', 'r' # Set column alignment
print(pt)

In [ ]:
import nltk
nltk.download('wordnet')

In [ ]:
from prettytable import PrettyTable

wnl = nltk.WordNetLemmatizer()
tokens_lower = [w.lower() for w in tokens]
text_lemmata = [wnl.lemmatize(t) for t in tokens_lower if t not in stop]


text_lemmata = filter(lambda word: word not in ',-:', text_lemmata)

pt = PrettyTable(field_names=['Lemma', 'Frequency']) 
c = Counter(text_lemmata)
[ pt.add_row(kv) for kv in c.most_common()[:20] ]
pt.align['Stem'], pt.align['Frequency'] = 'l', 'r' # Set column alignment
print(pt)

In [ ]:
text.concordance('learning')

In [ ]:
print(text.similar(""))

In [ ]:
print(len(abstract))
abstracts = list()
number = 1000 # number of abstracts to download

for l in abstract[:number]:
    raw = urllib2.urlopen("http://research.google.com" + l).read() 
    soup = BeautifulSoup(raw) # Seite in HTML-Parser einlesen
    abstracts = abstracts + [t.text.strip().lower() for t in (td.find('div') for td in soup.findAll('div', class_="maia-col-8")) if a] # Überschriften stehen in pub-title
   
textfile = "\n".join(abstracts)
tokens = nltk.word_tokenize(textfile)
text = nltk.Text(tokens)
text_clean = [word for word in text if word not in ',-:()']

print "Done."